In [28]:
import os
import h5py
import numpy as np

In [70]:
# vector files:
cnc_n_1_files = [(f"cnc_{n}_1.jld","/cnc_n_1") for n in range(1,4)]
stab_n_files = [(f"stab_{n}.jld","/stab_n") for n in range(1,4)]

# decomposition files
cnc_decomp_n_files = [(f"cnc_decomposition_{n}.jld","/x_value") for n in range(1,4)]
stab_decomp_n_files = [(f"stab_decomposition_{n}.jld","/x_value") for n in range(1,5)]

# all relevant data files:
data_file_names = cnc_n_1_files+stab_n_files+cnc_decomp_n_files+stab_decomp_n_files

In [94]:
# Find current path
current = os.getcwd()
data_path = os.path.join(current, "data")
data_files = []
# Loop through files in data_path
for filename in data_file_names:
    file_path = os.path.join(data_path, filename[0])
    with h5py.File(file_path, 'r') as f:
        # 'f' is an open HDF5 file object, you can inspect its keys, datasets, etc.
        # For example, to list all datasets:
        filename_prefix = filename[0][:-4]
        data = f[filename[1]][:]
        if len(data.dtype) > 1:
            data = data["num"]
        np.save("./data/"+filename_prefix,np.transpose(data))
        #data_files.append(data)

In [ ]:
# some manual data retrieval:
with h5py.File("./large_data/stab_4.jld", 'r') as f:
        # 'f' is an open HDF5 file object, you can inspect its keys, datasets, etc.
        # For example, to list all datasets:
        data = np.transpose(f['/stab_n'][:]["num"])
        np.save("./large_data/stab_4",data)
        #data_files.append(data)

In [103]:
# some manual data retrieval:
with h5py.File("./large_data/cnc_vectors_4_1.jld", 'r') as f:
        # 'f' is an open HDF5 file object, you can inspect its keys, datasets, etc.
        # For example, to list all datasets:
        data = np.transpose(f['/cnc_vectors_4_1'][:]["num"])
        np.save("./large_data/cnc_4_1",data)
        #data_files.append(data)

In [276]:
# some manual data retrieval:
cnc_support_4 = None
cnc_decomposition_4 = None
with h5py.File("./data/cnc_decomposition_4_1.jld", 'r') as f:
        # 'f' is an open HDF5 file object, you can inspect its keys, datasets, etc.
        # For example, to list all datasets:
        #data = f['/cnc_decomposition_4_1']
        key = list(f.keys())[1]

        # random sample of 80k cnc 4-1 vectors
        cnc_samples_4 = f[(f[key][()])[0]][:]
        # python indexing:
        cnc_samples_4 = [int(x-1) for x in cnc_samples_4]
        # optimal decomposition using random sample:
        cnc_decomposition_4 = f[(f[key][()])[1]][:]

In [266]:
def t_state(n):
    T = np.array([1.0,1/np.sqrt(2),1/np.sqrt(2),0.0])
    T_state = T
    for i in range(1,n):
        T_state = np.kron(T_state,T)
    return T_state

t_state(2)

array([1.        , 0.70710678, 0.70710678, 0.        , 0.70710678,
       0.5       , 0.5       , 0.        , 0.70710678, 0.5       ,
       0.5       , 0.        , 0.        , 0.        , 0.        ,
       0.        ])

# saving simulation keys

In [ ]:
# Find current path
current = os.getcwd()
data_path = os.path.join(current, "data")
data_files = dict()

for n in range(1,5):
    # generate filenames:
    if n == 5:
        vec_filename = "./large_data/cnc_4_1.npy"
    else:
        vec_filename = os.path.join(data_path,f"cnc_{n}_1.npy")
    supp_filename = os.path.join(data_path,f"cnc_decomposition_{n}.npy")

    # load data
    vec_data = np.load(vec_filename)
    supp_data = np.load(supp_filename)

    # threshold value:
    theshold = 10**(-(16))
    nonzero_support = [i for i in range(len(supp_data)) if abs(supp_data[i]) > theshold]
    data_files[n] = (supp_data[nonzero_support],vec_data[:,nonzero_support])
    #data_files.append((supp_data[nonzero_support],vec_data[:,nonzero_support]))
    #print(sum(supp_data[nonzero_support]))

In [291]:
# load 4-qubit data:
cnc_4_1 = np.load("./large_data/cnc_4_1.npy")

In [292]:
# take only non-negligible supports:
threshold = 10**(-16)
nonzero_support = [i for i in range(len(cnc_decomposition_4)) if abs(cnc_decomposition_4[i]) > threshold]

# extract optimal subset:
cnc_support_4 = [cnc_samples_4[i] for i in nonzero_support]
cnc_optimal_4_1 = cnc_4_1[:,cnc_support_4]
cnc_optimal_decomp_4 = cnc_decomposition_4[nonzero_support]

In [ ]:
# add 4-qubit data to dictionary:
data_files[4] = (cnc_optimal_decomp_4,cnc_optimal_4_1)

In [334]:
with h5py.File("./keys/all_cnc_keys_4.h5", "w") as f:
    for n in range(1,5):
        reshaped_vector = np.transpose(data_files[n][0]).reshape(len(data_files[n][0]),1)
        concatenated_array = np.concatenate((np.transpose(reshaped_vector),data_files[n][1]),axis = 0)
        f.create_dataset(f"n={n}", data=concatenated_array)

In [336]:
# Find current path
current = os.getcwd()
data_path = os.path.join(current, "data")
stab_data_files = dict()

for n in range(1,5):
    # generate filenames:
    if n == 4:
        vec_filename = "./large_data/stab_4.npy"
    else:
        vec_filename = os.path.join(data_path,f"stab_{n}.npy")
    supp_filename = os.path.join(data_path,f"stab_decomposition_{n}.npy")

    # load data
    vec_data = np.load(vec_filename)
    supp_data = np.load(supp_filename)

    # threshold value:
    theshold = 10**(-(16))
    nonzero_support = [i for i in range(len(supp_data)) if abs(supp_data[i]) > theshold]
    stab_data_files[n] = (supp_data[nonzero_support],vec_data[:,nonzero_support])
    #data_files.append((supp_data[nonzero_support],vec_data[:,nonzero_support]))
    #print(sum(supp_data[nonzero_support]))

In [343]:
with h5py.File("./keys/all_stab_keys_4.h5", "w") as f:
    for n in range(1,5):
        reshaped_vector = np.transpose(stab_data_files[n][0]).reshape(len(stab_data_files[n][0]),1)
        concatenated_array = np.concatenate((np.transpose(reshaped_vector),stab_data_files[n][1]),axis = 0)
        f.create_dataset(f"n={n}", data=concatenated_array)

In [344]:
# how to load data
with h5py.File("./keys/all_stab_keys_4.h5", "r") as f:
    n_data_set = f["n=1"][:]
n_data_set

array([[-0.20710678,  0.70710678,  0.5       ],
       [ 1.        ,  1.        ,  1.        ],
       [ 0.        ,  1.        ,  0.        ],
       [-1.        ,  0.        ,  1.        ],
       [ 0.        ,  0.        ,  0.        ]])